# Library

In [1]:
pip install --upgrade pip setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 70.0.0
    Uninstalling setuptools-70.0.0:
      Successfully uninstalled setuptools-70.0.0
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 24.5.0 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
jupyterlab 4.3.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= 

In [3]:
# Librerie standard
import os
import random
import time
import re
import shutil
from pathlib import Path
from collections import defaultdict, Counter
from itertools import islice
from concurrent.futures import ProcessPoolExecutor

# Librerie per il trattamento delle immagini
import cv2
import imageio.v3 as imageio
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Librerie per il machine learning e deep learning
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as TF
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Librerie per la gestione dei dati
import pandas as pd
import json
import orjson
import ast

# Librerie per il progresso e il monitoraggio
from tqdm import tqdm

import warnings

from collections import defaultdict
from sklearn.metrics import accuracy_score
import torch
import torch.optim as optim
from torch.amp import GradScaler, autocast
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# Path

In [4]:
#Output folders and file names
COCO_JSON_NM = 'COCO_annotations_new.json'
OUT_COCO_JSON_NM = 'mod_COCO_annotations_new.json'
OUT_IMAGE_FLDR_NM = 'images'
OUT_CFG_FLDR_NM = 'YOLO_cfg'
RANDOM_SEED = 2023

in_dataset_pth = Path('/kaggle/input/our-xview-dataset')
out_dataset_pth = Path('/kaggle/working/')
img_fldr = Path(f'/kaggle/input/our-xview-dataset/{OUT_IMAGE_FLDR_NM}')
cfg_fldr_pth = Path(f'/kaggle/input/our-xview-dataset/{OUT_CFG_FLDR_NM}')

coco_json_pth = in_dataset_pth / COCO_JSON_NM
new_coco_json_pth = out_dataset_pth / OUT_COCO_JSON_NM
train_txt_pth = cfg_fldr_pth / 'train.txt'
val_txt_pth = cfg_fldr_pth / 'val.txt'
test_txt_pth = cfg_fldr_pth / 'test.txt'

#DATASET
train_path = '/kaggle/working/train.json'
test_path = '/kaggle/working/test.json'
val_path = '/kaggle/working/val.json'

random.seed(RANDOM_SEED)

In [5]:
# Pulizia dell'output per cartelle specifiche
def clean_output(output_dir):
    if output_dir.exists() and output_dir.is_dir():
        for item in output_dir.iterdir():
            if item.is_dir():
                shutil.rmtree(item)  # Rimuove la sotto-cartella
            else:
                item.unlink()  # Rimuove il file
        print(f"Cartella {output_dir} pulita.")
    else:
        print(f"Cartella {output_dir} non trovata. Nessuna azione necessaria.")

# Pulisce la cartella di output prima di avviare il processo
clean_output(out_dataset_pth)

Cartella /kaggle/working pulita.


In [6]:
# Sopprime i warning specifici del modulo skimage
warnings.filterwarnings("ignore", 
    message="Applying `local_binary_pattern` to floating-point images may give unexpected results.*")

# Background Labels

In [7]:
def process_custom_coco_json(input_path, output_path):
    """
    Funzione per processare un JSON COCO in formato personalizzato.
    """
    # Leggi il JSON dal file di input
    with open(input_path, 'r') as f:
        data = json.load(f)

    # Ottieni e correggi il formato delle categorie
    raw_categories = data.get('categories', [])
    categories = []

    for category in tqdm(raw_categories, desc="Processing Categories"):
        for id_str, name in category.items():
            try:
                categories.append({"id": int(id_str), "name": name})
            except ValueError:
                print(f"Errore nel parsing della categoria: {category}")

    # Trova la categoria "Aircraft" con ID 0
    aircraft_category = next((cat for cat in categories if cat['id'] == 0 and cat['name'] == "Aircraft"), None)
    if aircraft_category:
        aircraft_category['id'] = 11  # Cambia l'ID della categoria "Aircraft" a 11

    # Aggiungi la categoria "background" con ID 0 se non esiste
    if not any(cat['id'] == 0 for cat in categories):
        categories.append({"id": 0, "name": "background"})

    # Preprocessa le annotazioni in un dizionario per immagini
    image_annotations_dict = {}
    for annotation in tqdm(data.get('annotations', []), desc="Building Image Annotations Dictionary"):
        image_id = annotation['image_id']
        if image_id not in image_annotations_dict:
            image_annotations_dict[image_id] = []
        image_annotations_dict[image_id].append(annotation)

    # Lista di nuove annotazioni da aggiungere per immagini senza bbox
    new_annotations = []

    # Elenco di annotazioni da rimuovere
    annotations_to_remove = []

    for annotation in tqdm(data.get('annotations', []), desc="Processing Annotations"):
        if annotation['category_id'] == 0:  # Se è Aircraft
            annotation['category_id'] = 11
        
        # Converte il formato del bbox
        if isinstance(annotation['bbox'], str):
            annotation['bbox'] = json.loads(annotation['bbox'])
        
        x, y, width, height = annotation['bbox']
        xmin = x
        xmax = x + width
        ymin = y
        ymax = y + height
        
        # Verifica che xmin < xmax e ymin < ymax
        if xmin >= xmax or ymin >= ymax:
            annotations_to_remove.append(annotation['id'])
        else:
            annotation['bbox'] = [xmin, xmax, ymin, ymax]

    # Rimuovi le annotazioni non valide
    data['annotations'] = [ann for ann in data['annotations'] if ann['id'] not in annotations_to_remove]

    # Verifica se ci sono immagini senza annotazioni (usando il dizionario delle annotazioni)
    for image in tqdm(data.get('images', []), desc="Processing Images"):
        if image['id'] not in image_annotations_dict:  # Se l'immagine non ha annotazioni
            # Aggiungi la categoria "background"
            new_annotation = {
                'id': len(data['annotations']) + len(new_annotations),
                'image_id': image['id'],
                'category_id': 0,  # Categoria background con ID 0
                'area': image['width'] * image['height'],
                'bbox': [0.0, image['width'], 0.0, image['height']],  # Background con bbox che copre tutta l'immagine
                'iscrowd': 0
            }
            new_annotations.append(new_annotation)

    # Aggiungi le nuove annotazioni al JSON originale
    data['annotations'].extend(new_annotations)

    # Aggiorna le categorie nel JSON
    data['categories'] = categories

    # Scrivi il JSON modificato nel file di output
    with open(output_path, 'w') as f:
        json.dump(data, f, indent=4)

In [8]:
process_custom_coco_json(coco_json_pth, new_coco_json_pth)

Processing Images: 100%|██████████| 45891/45891 [00:00<00:00, 847384.31it/s]


In [9]:
def count_bboxes_per_category(json_path):
    """
    Funzione che conta il numero di bounding box per ciascuna categoria in un file JSON formato COCO.
    
    :param json_path: Percorso al file JSON.
    :return: Dizionario con i nomi delle categorie come chiavi e il conteggio dei bounding box come valori.
    """
    # Leggi il JSON dal file
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Ottieni mapping delle categorie (id -> nome)
    category_mapping = {cat['id']: cat['name'] for cat in data.get('categories', [])}
    
    # Conta i bounding box per ciascun category_id
    bbox_counts = defaultdict(int)
    for annotation in data.get('annotations', []):
        category_id = annotation['category_id']
        bbox_counts[category_id] += 1
    
    # Converti il conteggio usando i nomi delle categorie
    bbox_counts_named = {category_mapping[cat_id]: count for cat_id, count in bbox_counts.items()}

    return bbox_counts_named

In [10]:
bbox_counts = count_bboxes_per_category(new_coco_json_pth)

# Stampa i risultati
for category, count in bbox_counts.items():
    print(f"Categoria: {category}, Numero di bbox: {count}")

Categoria: Building, Numero di bbox: 384942
Categoria: Passenger Vehicle, Numero di bbox: 225097
Categoria: Railway Vehicle, Numero di bbox: 4233
Categoria: Truck, Numero di bbox: 34377
Categoria: Aircraft, Numero di bbox: 1708
Categoria: Engineering Vehicle, Numero di bbox: 5473
Categoria: Storage Tank, Numero di bbox: 2033
Categoria: Shipping Container, Numero di bbox: 5391
Categoria: Maritime Vessel, Numero di bbox: 6329
Categoria: Pylon, Numero di bbox: 470
Categoria: Helipad, Numero di bbox: 152
Categoria: background, Numero di bbox: 13692


# Splitting

In [11]:
def split(json_file, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # Carica il JSON
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Ottieni la lista delle immagini
    images = data['images']
    
    # Mescola casualmente gli ID delle immagini
    random.shuffle(images)
    
    # Calcola i limiti per train, validation, e test
    total_images = len(images)
    total_annotations = len(data['annotations'])
    train_end = int(total_images * train_ratio)
    val_end = int(total_images * (train_ratio + val_ratio))
    
    # Suddividi le immagini nei rispettivi set
    train_images = images[:train_end]
    val_images = images[train_end:val_end]
    test_images = images[val_end:]
    
    # Raggruppa gli ID delle immagini per i rispettivi set
    train_image_ids = {image['id'] for image in train_images}
    val_image_ids = {image['id'] for image in val_images}
    test_image_ids = {image['id'] for image in test_images}
    
    # Filtra le annotazioni per i rispettivi set di immagini
    train_annotations = []
    val_annotations = []
    test_annotations = []
    
    for annotation in data['annotations']:
        if annotation['image_id'] in train_image_ids:
            train_annotations.append(annotation)
        elif annotation['image_id'] in val_image_ids:
            val_annotations.append(annotation)
        elif annotation['image_id'] in test_image_ids:
            test_annotations.append(annotation)
    
    # Crea i nuovi JSON per train, validation, e test
    train_data = {'images': train_images, 'annotations': train_annotations, 'categories': data['categories']}
    val_data = {'images': val_images, 'annotations': val_annotations, 'categories': data['categories']}
    test_data = {'images': test_images, 'annotations': test_annotations, 'categories': data['categories']}
    
    # Salva i file JSON
    with open('train.json', 'w') as f:
        json.dump(train_data, f, indent=4)
    
    with open('val.json', 'w') as f:
        json.dump(val_data, f, indent=4)
    
    with open('test.json', 'w') as f:
        json.dump(test_data, f, indent=4)
    
    # Controlla la proporzione delle immagini e delle annotazioni
    check_split_proportions(total_images, total_annotations, len(train_images), len(val_images), len(test_images), 
                            len(train_annotations), len(val_annotations), len(test_annotations), 
                            train_ratio, val_ratio, test_ratio, train_annotations, val_annotations, test_annotations, data['categories'])


def check_split_proportions(total_images, total_annotations, train_count, val_count, test_count, 
                            train_bbox_count, val_bbox_count, test_bbox_count, 
                            train_ratio, val_ratio, test_ratio, 
                            train_annotations, val_annotations, test_annotations, categories):
    # Percentuali per immagini
    train_image_percentage = (train_count / total_images) * 100
    val_image_percentage = (val_count / total_images) * 100
    test_image_percentage = (test_count / total_images) * 100
    
    # Percentuali per bbox
    train_bbox_percentage = (train_bbox_count / total_annotations) * 100
    val_bbox_percentage = (val_bbox_count / total_annotations) * 100
    test_bbox_percentage = (test_bbox_count / total_annotations) * 100
    
    print(f"Totale immagini: {total_images}")
    print(f"Totale annotazioni (bbox): {total_annotations}")
    print(f"Train: {train_count} immagini ({train_image_percentage:.2f}%) ({train_bbox_count} bbox) ({train_bbox_percentage:.2f}%)")
    print(f"Val: {val_count} immagini ({val_image_percentage:.2f}%) ({val_bbox_count} bbox) ({val_bbox_percentage:.2f}%)")
    print(f"Test: {test_count} immagini ({test_image_percentage:.2f}%) ({test_bbox_count} bbox) ({test_bbox_percentage:.2f}%)")
    
    # Calcola il numero di annotazioni per categoria nei vari set
    category_count_train = defaultdict(int)
    category_count_val = defaultdict(int)
    category_count_test = defaultdict(int)
    
    for annotation in train_annotations:
        category_count_train[annotation['category_id']] += 1
    for annotation in val_annotations:
        category_count_val[annotation['category_id']] += 1
    for annotation in test_annotations:
        category_count_test[annotation['category_id']] += 1
    
    # Stampa le proporzioni per categoria
    print("\nProporzioni per categoria:")
    for category in categories:
        category_id = category['id']
        category_name = category['name']
        
        # Conta il numero di annotazioni per categoria in ogni set
        train_cat_count = category_count_train.get(category_id, 0)
        val_cat_count = category_count_val.get(category_id, 0)
        test_cat_count = category_count_test.get(category_id, 0)
        
        # Calcola la percentuale di annotazioni per categoria
        total_cat_annotations = train_cat_count + val_cat_count + test_cat_count
        if total_cat_annotations > 0:
            train_cat_percentage = (train_cat_count / total_cat_annotations) * 100
            val_cat_percentage = (val_cat_count / total_cat_annotations) * 100
            test_cat_percentage = (test_cat_count / total_cat_annotations) * 100
        else:
            train_cat_percentage = val_cat_percentage = test_cat_percentage = 0.0
        
        print(f"{category_name}:")
        print(f"  Train: {train_cat_count} annotazioni ({train_cat_percentage:.2f}%)")
        print(f"  Val: {val_cat_count} annotazioni ({val_cat_percentage:.2f}%)")
        print(f"  Test: {test_cat_count} annotazioni ({test_cat_percentage:.2f}%)")

In [12]:
# Chiamata della funzione
split(new_coco_json_pth)

Totale immagini: 45891
Totale annotazioni (bbox): 683897
Train: 36712 immagini (80.00%) (544927 bbox) (79.68%)
Val: 4589 immagini (10.00%) (67281 bbox) (9.84%)
Test: 4590 immagini (10.00%) (71689 bbox) (10.48%)

Proporzioni per categoria:
Aircraft:
  Train: 1335 annotazioni (78.16%)
  Val: 194 annotazioni (11.36%)
  Test: 179 annotazioni (10.48%)
Passenger Vehicle:
  Train: 177753 annotazioni (78.97%)
  Val: 23340 annotazioni (10.37%)
  Test: 24004 annotazioni (10.66%)
Truck:
  Train: 27410 annotazioni (79.73%)
  Val: 3489 annotazioni (10.15%)
  Test: 3478 annotazioni (10.12%)
Railway Vehicle:
  Train: 3248 annotazioni (76.73%)
  Val: 525 annotazioni (12.40%)
  Test: 460 annotazioni (10.87%)
Maritime Vessel:
  Train: 5131 annotazioni (81.07%)
  Val: 570 annotazioni (9.01%)
  Test: 628 annotazioni (9.92%)
Engineering Vehicle:
  Train: 4379 annotazioni (80.01%)
  Val: 526 annotazioni (9.61%)
  Test: 568 annotazioni (10.38%)
Building:
  Train: 308131 annotazioni (80.05%)
  Val: 36460 anno

# DataLoader

In [13]:
class CustomDataset(Dataset):
    def __init__(self, coco_json_file, img_dir, aug=False):
        """
        Inizializza il dataset personalizzato.
        Args:
        - coco_json_file: Il file JSON contenente le annotazioni.
        - img_dir: La cartella delle immagini.
        - aug: Booleano per attivare o meno l'augmentazione.
        """
        # Carica il file JSON delle annotazioni
        with open(coco_json_file, 'r') as f:
            coco_data = json.load(f)

        # Crea una struttura per le annotazioni
        self.image_annotations = {}
        self.image_bboxes = {}

        # Estrai le classi (categorie) dal file JSON
        self.classes = {}
        for category in coco_data['categories']:
            self.classes[category['id']] = category['name']

        # Aggiungi la mappa di annotazioni valide
        for annotation in coco_data['annotations']:
            image_id = annotation['image_id']
            category_id = annotation['category_id']
            bbox = annotation['bbox']  # Formato [x_min, y_min, width, height]
            x_min, y_min, width, height = bbox
            x_max = x_min + width
            y_max = y_min + height
            # Modifica il formato del bbox in [x_min, x_max, y_min, y_max]
            annotation['bbox'] = [x_min, x_max, y_min, y_max]

            if image_id not in self.image_annotations:
                self.image_annotations[image_id] = []
                self.image_bboxes[image_id] = []

            self.image_annotations[image_id].append(category_id)
            self.image_bboxes[image_id].append(annotation['bbox'])

        # Mappa per associare ID immagine a file_name
        self.image_info = {
            image['id']: image['file_name']
            for image in coco_data['images']
        }

        # Filtra le immagini con annotazioni valide
        self.img_dir = img_dir
        self.image_paths = []
        self.image_ids = []
        for image_id, file_name in self.image_info.items():
            if image_id in self.image_annotations:  # Considera solo immagini con annotazioni
                img_path = os.path.join(img_dir, file_name)
                if os.path.exists(img_path):  # Assicura che il file esista
                    self.image_paths.append(img_path)
                    self.image_ids.append(image_id)

        # Trasformazioni di base e di augmentation
        self.base_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

        self.aug_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

        self.aug = aug

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        # Estrai il nome dell'immagine e l'ID corrispondente
        img_path = self.image_paths[index]
        img_id = self.image_ids[index]

        # Carica l'immagine
        image = Image.open(img_path).convert('RGB')
        original_width, original_height = image.size

        # Applica le trasformazioni
        if self.aug:
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)

        # Estrai le annotazioni e i bounding boxes
        categories = self.image_annotations.get(img_id, [])
        bboxes = self.image_bboxes.get(img_id, [])

        # Controllo di fallback per annotazioni mancanti
        if not bboxes:  
            raise ValueError(f"L'immagine {img_path} è stata erroneamente inclusa senza annotazioni valide.")

        # Converte da formato [x_min, x_max, y_min, y_max] a formato tensor
        scale_x = 224 / original_width
        scale_y = 224 / original_height

        # Scaling dei bounding boxes
        scaled_bboxes = []
        for bbox in bboxes:
            x_min, x_max, y_min, y_max = bbox

            scaled_bboxes.append(torch.tensor([
                float(x_min) * scale_x,  # x_min
                float(x_max) * scale_x,  # x_max
                float(y_min) * scale_y,  # y_min
                float(y_max) * scale_y   # y_max
            ], dtype=torch.float32))

        target = {
            "boxes": torch.stack(scaled_bboxes),
            "labels": torch.tensor(categories, dtype=torch.int64)
        }

        return image_tensor, target

In [14]:
def collate_fn(batch):
    """
    Funzione di collation per il DataLoader, utile per il batching di immagini e annotazioni.
    La funzione restituirà un batch di immagini e un batch di target, formattato correttamente per Faster R-CNN.
    
    Args:
    - batch: lista di tuple (image, target)
    
    Returns:
    - images: batch di immagini
    - targets: lista di dizionari contenenti le annotazioni per ogni immagine
    """
    # Separa immagini e target
    images, targets = zip(*batch)

    # Converte la lista di immagini in un batch di immagini
    images = list(images)

    # Restituisci il batch
    return images, list(targets)

In [15]:
# Creazione dei dataset
train_dataset = CustomDataset(train_path, img_fldr,  aug=True)
valid_dataset = CustomDataset(val_path, img_fldr, aug=False)  
test_dataset = CustomDataset(test_path, img_fldr, aug=False)  

# Creazione dei DataLoader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn, num_workers=2, pin_memory=True)
val_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn, num_workers=2, pin_memory=True)

## Check DataLoader

In [15]:
def validate_dataloader(dataloader):
    """
    Valida un DataLoader verificando che ogni immagine abbia un target associato
    e che nessun target sia `None` o vuoto.
    
    Args:
    - dataloader: Il DataLoader da verificare.
    
    Returns:
    - error_messages: Lista di messaggi di errore. Vuota se tutti i dati sono validi.
    """
    error_messages = []
    for batch_idx, (images, targets) in enumerate(dataloader):
        for idx, target in enumerate(targets):
            if target is None:
                error_messages.append(f"Batch {batch_idx}, Immagine {idx}: Target è None.")
            elif target["boxes"].numel() == 0 or target["labels"].numel() == 0:
                error_messages.append(
                    f"Batch {batch_idx}, Immagine {idx}: Target è vuoto o mancano 'boxes'/'labels'."
                )
    return error_messages

In [16]:
# Validazione del DataLoader di training
train_errors = validate_dataloader(train_loader)

if train_errors:
    print("Errori nel DataLoader di training:")
    for error in train_errors:
        print(error)
else:
    print("Tutti i target nel DataLoader di training sono validi.")

Tutti i target nel DataLoader di training sono validi.


In [17]:
# Validazione del DataLoader di training
val_errors = validate_dataloader(val_loader)

if val_errors:
    print("Errori nel DataLoader di validation:")
    for error in val_errors:
        print(error)
else:
    print("Tutti i target nel DataLoader di validation sono validi.")

Tutti i target nel DataLoader di validation sono validi.


In [18]:
# Validazione del DataLoader di training
test_errors = validate_dataloader(test_loader)

if test_errors:
    print("Errori nel DataLoader di test:")
    for error in test_errors:
        print(error)
else:
    print("Tutti i target nel DataLoader di test sono validi.")

Tutti i target nel DataLoader di test sono validi.


In [19]:
def count_images_and_targets(dataloader):
    """
    Conta il numero totale di immagini e target in un DataLoader.
    """
    num_images = 0
    num_targets = 0

    for images, targets in dataloader:
        # Conta le immagini nel batch
        num_images += len(images)
        
        # Conta i target per ogni immagine (numero di oggetti)
        for target in targets:
            num_targets += len(target["boxes"])  # Ogni immagine ha un numero di bounding boxes
    
    return num_images, num_targets

In [20]:
num_images_train, num_targets_train = count_images_and_targets(train_loader)

print(f"Numero totale di immagini per il train: {num_images_train}")
print(f"Numero totale di target per il train: {num_targets_train}")

Numero totale di immagini per il train: 36686
Numero totale di target per il train: 544927


In [21]:
num_images_val, num_targets_val = count_images_and_targets(val_loader)

print(f"Numero totale di immagini per il validation: {num_images_val}")
print(f"Numero totale di target per il validation: {num_targets_val}")

Numero totale di immagini per il validation: 4585
Numero totale di target per il validation: 67281


In [22]:
num_images_test, num_targets_test = count_images_and_targets(test_loader)

print(f"Numero totale di immagini per il test: {num_images_test}")
print(f"Numero totale di target per il test: {num_targets_test}")

Numero totale di immagini per il test: 4583
Numero totale di target per il test: 71689


In [23]:
print(f"Numero totale di immagini: {num_images_train + num_images_val +num_images_test}")
print(f"Numero totale di target: {num_targets_train + num_targets_val +num_targets_test}")

Numero totale di immagini: 45854
Numero totale di target: 683897


# Modello Faster R-CNN (Resnet50)

In [16]:
def compute_class_weights(dataset):
    # Conta la frequenza di ogni classe nel dataset
    class_counts = np.zeros(len(dataset.classes))  
    
    # Usa tqdm per monitorare il progresso mentre si itera sul dataset
    for _, targets in tqdm(dataset, desc="Calcolo frequenze delle classi", leave=False):
        # Controlla se targets è None
        if targets is None:
            print("ERRORE TARGET NONE")
            continue
        
        # Assicurati che 'labels' sia un array e itera su di esso
        if 'labels' in targets:
            for target in targets['labels']:  
                class_counts[target] += 1

    # Calcola i pesi per le classi 
    total_count = sum(class_counts)  

    # Calcola i pesi inversamente proporzionali alla frequenza
    class_weights = np.divide(total_count, class_counts)

    return class_weights

In [118]:
def train_and_validate(model, train_loader, val_loader, optimizer, device, class_weights=None, 
                       num_epochs=10, num_classes=12, accumulation_steps=4):
    """
    Funzione di training e validazione per Faster R-CNN.
    """
    # Scaler per mixed precision training
    scaler = GradScaler()
    model.to(device)

    train_losses = []
    val_losses = []
    train_mAPs = []
    val_mAPs = []

    # Converte class_weights in tensor e sposta sul dispositivo
    if class_weights is not None:
        class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

    for epoch in range(num_epochs):
        print(f"\nEpoca {epoch + 1}/{num_epochs}")

        # --------------------
        # Training
        # --------------------
        model.train()
        total_train_loss = 0.0
        all_train_preds = []
        all_train_targets = []

        optimizer.zero_grad()
        train_loop = tqdm(train_loader, desc="Training", leave=False)

        for i, (images, targets) in enumerate(train_loop):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
            with autocast('cuda'):
                # Forward pass
                loss_dict = model(images, targets)
            
                total_weighted_loss = 0
                total_weight = 0
            
                # Itera su tutte le perdite
                for key, loss in loss_dict.items():
                    if isinstance(loss, torch.Tensor):
                        if key == 'loss_classifier' and class_weights is not None:
                            # Applica i pesi delle classi solo alla loss_classifier
                            labels = torch.cat([t['labels'] for t in targets])  # Ottieni le etichette
                            weighted_loss = loss * class_weights[labels]  # Pesi basati sulle etichette
                            total_weighted_loss += weighted_loss.sum()  # Somma la perdita pesata
                            total_weight += class_weights[labels].sum()  # Somma i pesi per il calcolo totale
                        else:
                            # Entra qui per loss_box_reg, loss_objectness e loss_rpn_box_reg
                            total_weighted_loss += loss.sum()  # Somma la perdita normale
                            total_weight += loss.numel()  # Somma il numero di elementi
            
                losses = total_weighted_loss / total_weight  # Calcola la perdita totale normalizzata

            # Backward pass con gradient scaling
            scaler.scale(losses).backward()
        
            # Gradient accumulation
            if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
        
            total_train_loss += losses.item() * accumulation_steps
            train_loop.set_postfix(loss=losses.item() * accumulation_steps)
            
            # Raccoglie predizioni per calcolo mAP
            model.eval()
            with torch.no_grad():
                outputs = model(images)
                all_train_preds.extend([{k: v.cpu() for k, v in t.items()} for t in outputs])
                all_train_targets.extend([{k: v.cpu() for k, v in t.items()} for t in targets])
            model.train()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"Perdita media di training: {avg_train_loss:.4f}")

        train_map = calculate_map(all_train_preds, all_train_targets, num_classes=num_classes)
        train_mAPs.append(train_map)
        print(f"mAP di training: {train_map:.4f}")

        # --------------------
        # Validazione
        # --------------------
        model.eval()
        total_val_loss = 0.0
        all_val_preds = []
        all_val_targets = []
        
        val_loop = tqdm(val_loader, desc="Validazione", leave=False)
        with torch.no_grad():
            for images, targets in val_loop:
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
                with autocast('cuda'):
                    # Forward pass
                    loss_dict = model(images, targets)
        
                    total_weighted_loss = 0
                    total_weight = 0
                    for key, loss in loss_dict.items():
                        if isinstance(loss, torch.Tensor):
                            if key == 'loss_classifier' and class_weights is not None:
                                # Applica i pesi delle classi solo alla loss_classifier
                                labels = torch.cat([t['labels'] for t in targets])  # Ottieni le etichette
                                weighted_loss = loss * class_weights[labels]  # Pesi basati sulle etichette
                                total_weighted_loss += weighted_loss.sum()  # Somma la perdita pesata
                                total_weight += class_weights[labels].sum()  # Somma i pesi per il calcolo totale
                            else:
                                total_weighted_loss += loss.sum()  # Somma la perdita normale
                                total_weight += loss.numel()  # Somma il numero di elementi
    
                    losses = total_weighted_loss / total_weight  # Calcola la perdita totale normalizzata
        
                total_val_loss += losses.item()
        
                # Predizioni per calcolo mAP
                outputs = model(images)
                all_val_preds.extend([{k: v.cpu() for k, v in t.items()} for t in outputs])
                all_val_targets.extend([{k: v.cpu() for k, v in t.items()} for t in targets])
        
        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Perdita media di validazione: {avg_val_loss:.4f}")
        
        val_map = calculate_map(all_val_preds, all_val_targets, num_classes=num_classes)
        val_mAPs.append(val_map)
        print(f"mAP di validazione: {val_map:.4f}")

        # --------------------
        # Salvataggio del modello
        # --------------------
        torch.save(model.state_dict(), f"model_epoch_{epoch + 1}.pth")
        print(f"Modello salvato: model_epoch_{epoch + 1}.pth")

    return train_losses, val_losses, train_mAPs, val_mAPs

In [18]:
def plot_metrics(losses_per_epoch, train_accuracies, val_accuracies, num_epochs):
    """
    Funzione per plottare le metriche di training e validazione.
    """
    epochs_range = range(1, num_epochs + 1)

    plt.figure(figsize=(12, 8))

    # Loss plot
    plt.subplot(2, 1, 1)
    plt.plot(epochs_range, losses_per_epoch, label='Training Loss', color='blue')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training Loss per Epoca')

    # Accuracy plot
    plt.subplot(2, 1, 2)
    plt.plot(epochs_range, train_accuracies, label='Accuracy (Training)', color='green')
    plt.plot(epochs_range, val_accuracies, label='Accuracy (Validation)', color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy per Epoca')

    plt.tight_layout()
    plt.show()

In [19]:
def test_model(model, test_loader, device):
    """
    Funzione per il testing del modello Faster R-CNN.
    
    Args:
    - model: il modello Faster R-CNN.
    - test_loader: DataLoader per il test set.
    - device: dispositivo su cui eseguire (es. 'cuda' o 'cpu').
    
    Returns:
    - predictions: lista delle predizioni per ogni batch (include 'boxes', 'labels', 'scores').
    """
    model.to(device)
    model.eval()
    predictions = []
    
    print("\nInizio testing...")
    test_loop = tqdm(test_loader, desc="Testing", leave=False)
    
    with torch.no_grad():
        for images, _ in test_loop:  # Durante il test, i target possono essere ignorati
            images = [img.to(device) for img in images]
            preds = model(images)
            
            # Predizioni di ciascun batch (contenente 'boxes', 'labels', 'scores')
            # Le predizioni sono in un formato di lista di dizionari
            for pred in preds:
                predictions.append({
                    'boxes': pred['boxes'].cpu().numpy(),
                    'labels': pred['labels'].cpu().numpy(),
                    'scores': pred['scores'].cpu().numpy()
                })
            
            # Aggiungi aggiornamenti su quante predizioni sono state processate
            test_loop.set_postfix(processed=len(predictions))

    print("Testing completato.")
    return predictions

In [20]:
# Carica il modello Faster R-CNN con ResNet50 e FPN
model = fasterrcnn_resnet50_fpn(weights=None)

num_classes = 12

# Modifica il numero di classi in output
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Congela i layer della backbone (ResNet50)
for param in model.backbone.parameters():
    param.requires_grad = False

# Imposta il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Configurazione training
num_epochs = 2
optimizer = optim.AdamW(model.parameters(), lr=1e-7)

# Sposta il modello su GPU o CPU
model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 211MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [21]:
# Calcola i pesi delle classi
class_weights = compute_class_weights(train_loader.dataset)

print(class_weights)

[4.95748726e+01 3.06564165e+00 1.98805910e+01 1.67773091e+02
 1.06202884e+02 1.24440968e+02 1.76849132e+00 4.39457258e+03
 3.32272561e+02 1.23790777e+02 1.42651047e+03 4.08185019e+02]


In [116]:
train_losses, val_losses, train_mAPs, val_mAPs = train_and_validate(model, train_loader, val_loader, optimizer, device, class_weights, num_epochs)


Epoca 1/2


Training:   0%|          | 0/18343 [00:00<?, ?it/s, loss=13.6]

Class weights: tensor([4.9575e+01, 3.0656e+00, 1.9881e+01, 1.6777e+02, 1.0620e+02, 1.2444e+02,
        1.7685e+00, 4.3946e+03, 3.3227e+02, 1.2379e+02, 1.4265e+03, 4.0819e+02],
       device='cuda:0')
Shape of class_weights: torch.Size([12])
Labels for loss_classifier: tensor([1, 1, 1, 6, 1, 1, 1, 1, 6, 6], device='cuda:0')
Class weights for classifier: tensor([3.0656, 3.0656, 3.0656, 1.7685, 3.0656, 3.0656, 3.0656, 3.0656, 1.7685,
        1.7685], device='cuda:0')
Labels for loss_classifier: tensor([1, 1, 1, 6, 1, 1, 1, 1, 6, 6], device='cuda:0')
Class weights for classifier: tensor([3.0656, 3.0656, 3.0656, 1.7685, 3.0656, 3.0656, 3.0656, 3.0656, 1.7685,
        1.7685], device='cuda:0')
Labels for loss_classifier: tensor([1, 1, 1, 6, 1, 1, 1, 1, 6, 6], device='cuda:0')
Class weights for classifier: tensor([3.0656, 3.0656, 3.0656, 1.7685, 3.0656, 3.0656, 3.0656, 3.0656, 1.7685,
        1.7685], device='cuda:0')
Labels for loss_classifier: tensor([1, 1, 1, 6, 1, 1, 1, 1, 6, 6], device='

Training:   0%|          | 1/18343 [00:00<2:33:53,  1.99it/s, loss=8.14]

Class weights: tensor([4.9575e+01, 3.0656e+00, 1.9881e+01, 1.6777e+02, 1.0620e+02, 1.2444e+02,
        1.7685e+00, 4.3946e+03, 3.3227e+02, 1.2379e+02, 1.4265e+03, 4.0819e+02],
       device='cuda:0')
Shape of class_weights: torch.Size([12])
Labels for loss_classifier: tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 0],
       device='cuda:0')
Class weights for classifier: tensor([ 1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,
         1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,
         1.7685,  1.7685,  1.7685,  1.7685,  

...

Class weights: tensor([4.9575e+01, 3.0656e+00, 1.9881e+01, 1.6777e+02, 1.0620e+02, 1.2444e+02,
        1.7685e+00, 4.3946e+03, 3.3227e+02, 1.2379e+02, 1.4265e+03, 4.0819e+02],
       device='cuda:0')
Shape of class_weights: torch.Size([12])
Labels for loss_classifier: tensor([1, 6, 6, 6, 6, 6, 6, 6, 6, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 6, 6, 6],
       device='cuda:0')
Class weights for classifier: tensor([ 3.0656,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,
         1.7685, 19.8806,  3.0656,  3.0656,  3.0656,  3.0656, 19.8806, 19.8806,
        19.8806, 19.8806, 19.8806,  1.7685,  1.7685,  1.7685], device='cuda:0')
Labels for loss_classifier: tensor([1, 6, 6, 6, 6, 6, 6, 6, 6, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 6, 6, 6],
       device='cuda:0')
Class weights for classifier: tensor([ 3.0656,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,
         1.7685, 19.8806,  3.0656,  3.0656,  3.0656,  3.0656, 19.8806, 19.8806,
        19.8806, 19.8806, 19.8806,  1.7685,  

Training:   0%|          | 46/18343 [00:14<1:36:51,  3.15it/s, loss=8.43]

Class weights: tensor([4.9575e+01, 3.0656e+00, 1.9881e+01, 1.6777e+02, 1.0620e+02, 1.2444e+02,
        1.7685e+00, 4.3946e+03, 3.3227e+02, 1.2379e+02, 1.4265e+03, 4.0819e+02],
       device='cuda:0')
Shape of class_weights: torch.Size([12])
Labels for loss_classifier: tensor([6, 6, 6, 6, 6, 6, 6, 1, 1, 2, 1, 1, 6], device='cuda:0')
Class weights for classifier: tensor([ 1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  3.0656,
         3.0656, 19.8806,  3.0656,  3.0656,  1.7685], device='cuda:0')
Labels for loss_classifier: tensor([6, 6, 6, 6, 6, 6, 6, 1, 1, 2, 1, 1, 6], device='cuda:0')
Class weights for classifier: tensor([ 1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  3.0656,
         3.0656, 19.8806,  3.0656,  3.0656,  1.7685], device='cuda:0')
Labels for loss_classifier: tensor([6, 6, 6, 6, 6, 6, 6, 1, 1, 2, 1, 1, 6], device='cuda:0')
Class weights for classifier: tensor([ 1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  3.0656,
        

Class weights: tensor([4.9575e+01, 3.0656e+00, 1.9881e+01, 1.6777e+02, 1.0620e+02, 1.2444e+02,
        1.7685e+00, 4.3946e+03, 3.3227e+02, 1.2379e+02, 1.4265e+03, 4.0819e+02],
       device='cuda:0')
Shape of class_weights: torch.Size([12])
Labels for loss_classifier: tensor([6, 1, 6, 1, 2, 6, 6, 6, 6, 6, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6], device='cuda:0')
Class weights for classifier: tensor([ 1.7685,  3.0656,  1.7685,  3.0656, 19.8806,  1.7685,  1.7685,  1.7685,
         1.7685,  1.7685,  3.0656,  3.0656,  3.0656,  1.7685,  1.7685,  1.7685,
         1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,
         1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,
         1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,
         1.7685,  3.0656,  3.0656,  1.7685,  1.7685,  1.7685,  1.7685,  1.7685,
         1.7685,  1.

KeyboardInterrupt: 

In [ ]:
plot_metrics(losses, train_accs, val_accs, num_epochs)

In [ ]:
# Chiamata alla funzione di test
predictions = test_model(model=model, test_loader=test_loader, device=device)

# Puoi fare qualcosa con le predizioni, come visualizzarle o calcolare metriche
print(f"Numero di predizioni ottenute: {len(predictions)}")